<a href="https://colab.research.google.com/github/june1st/GoogleColab/blob/master/BERT_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTのテスト

<table class="tfo-notebook-buttons" align="left" >
 <td>
<img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
  <td>
<img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
  </td>
</table>
※下記の記事を参考に実行
https://qiita.com/Kosuke-Szk/items/4b74b5cce84f423b7125


In [0]:
! wget https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2

--2018-11-12 10:29:55--  https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2766642154 (2.6G) [application/octet-stream]
Saving to: ‘jawiki-latest-pages-articles.xml.bz2’

iki-latest-pages-ar  35%[======>             ] 943.38M  1.97MB/s    eta 14m 21s

In [0]:
! git clone https://github.com/attardi/wikiextractor.git

In [0]:
! python wikiextractor/WikiExtractor.py -o extracted jawiki-latest-pages-articles.xml.bz2

In [0]:
import glob
from bs4 import BeautifulSoup

with open('./tmp.txt','w') as f:
    for directory in glob.glob('./extracted/*'):
        for name in glob.glob(directory+'/*'):
            with open(name, 'r') as r:
                for line in r:
                    # titleを削除する
                    if '<doc ' in line:
                        next(r)
                        next(r)
                    elif '</doc>' in line:
                        f.write('\n')
                        continue
                    else:
                        # 空白・改行削除、大文字を小文字に変換
                        text = BeautifulSoup(line.strip()).text.lower()
                        f.write(text)

In [0]:
import linecache
import random
import MeCab

In [0]:
random.seed(42)
filename = 'tmp.txt'
save_file = 'even_rows100M.txt'
LIMIT_BYTE = 100000000 # 100Mbyte
# t = MeCab.Tagger('-Owakati') # Neologdを辞書に使っている人場合はそちらを使用するのがベターです
t = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/ -Owakati')

def get_byte_num(s):
    return len(s.encode('utf-8'))

In [0]:
with open(save_file, 'w') as f:
    count_byte = 0
    with open(filename) as r:
        for text in r:
            print('{} bytes'.format(count_byte))
            text = t.parse(text).strip()
            # 一文ごとに分割する
            text = text.split('。')
            # 空白要素は捨てる
            text = [t.strip() for t in text if t]
            # 一単元の文書が偶数個の文章から成るようにする(BERTのデータセットの都合上)
            max_text_len = len(text) // 2
            text = text[:max_text_len * 2]
            text = '\n'.join(text)
            f.write(text)
            count_byte += get_byte_num(text)
            if count_byte >= LIMIT_BYTE:
                break

In [0]:
num_lines = sum(1 for line in open(save_file))
print('Base file lines : ', num_lines)
# 全体の80%をTraining dataに当てます
train_lines = int(num_lines * 0.8)
print('Train file lines : ', train_lines)

In [0]:
! mkdir -p data output

In [0]:
out_file_name_temp = './data/splitted_%d.txt'

split_index = 1
line_index = 1
out_file = open(out_file_name_temp % (split_index,), 'w')
in_file = open(save_file)
line = in_file.readline()
while line:
    if line_index > train_lines:
        print('Starting file: %d' % split_index)
        out_file.close()
        split_index = split_index + 1
        line_index = 1
        out_file = open(out_file_name_temp % (split_index,), 'w')
    out_file.write(line)
    line_index = line_index + 1
    line = in_file.readline()
    
out_file.close()
in_file.close()

In [0]:
print('Train file lines : ', sum(1 for line in open('./data/splitted_1.txt')))
print('Valid file lines : ', sum(1 for line in open('./data/splitted_2.txt')))

前処理ここまで